In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt


pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
PROTEIN_SEQ_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')

In [2]:
pd.read_pickle(PROTEIN_SEQ_PATH)

,gene_symbol_harmonized,db,UniqueIdentifier,EntryName,ProteinName,OS,OX,GN,PE,SV,motifs,seq
0,Hsf1,tr,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor protein 1,Mus musculus,10090,Hsf1,1,1,NaN,MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHVFDQGQFAKEVLPKYFKHNNMASFVRQLNMYGFRKVVHIEQGGLVKPERDD...
1,Auts2,sp,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Mus musculus,10090,Auts2,1,2,NaN,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAPSLASSSGSDKEDNGKPPSSAPSRPRPPRRKRRESTSAEEDIIDGFAMTSF...
2,Ptgs2,tr,A0A087WPT2,A0A087WPT2_MOUSE,Prostaglandin G/H synthase 2 (Fragment),Mus musculus,10090,Ptgs2,4,1,NaN,MSTGFDQYKCDCTRTGFYGENCTTPEFLTRIKLLLKPTPNTVHYILTHFKGVWNIVNNIPFL
3,Fat1,tr,A0A087WPU4,A0A087WPU4_MOUSE,FAT atypical cadherin 1 (Fragment),Mus musculus,10090,Fat1,1,1,NaN,XRPYFDSKLNKNIYSDIPPQVPVRPISYTPSIPSDSRNNLDRNSFEGSAIPEHPEFSTFNPESMHGHRKAVAVCSVAPNLPPPPPSNSPSDSDSIQ...
4,Gm20905,tr,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814",Mus musculus,10090,Gm20905,3,1,NaN,MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFDKNENISPQAEADEDMGDEVDSMLDKSEVNNPAIGKDENISPQVKGDEDMG...
...,...,...,...,...,...,...,...,...,...,...,...,...
94781,Snph,tr,Z4YN82,Z4YN82_MOUSE,Syntaphilin (Fragment),Mus musculus,10090,Snph,1,1,NaN,MAMSLQGSRRASAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSCKGSDSSPTPRRSMKYTLCSDNHGIKPPTPEQYLTPLQQKEVCIR
94782,Timm22,tr,Z4YN86,Z4YN86_MOUSE,Mitochondrial import inner membrane translocase subunit Tim22 (Fragment),Mus musculus,10090,Timm22,1,1,NaN,MAATAPKAGGSAPEAAGSAEAPLQYSLLLQYLVGDKRQPRLLEPGSLGGIPSPAKSEEQKMIERAMESCAFKAVLACVGVPGKVGLEEQRHQWLHH...
94783,Akap17b,tr,Z4YN92,Z4YN92_MOUSE,A-kinase anchor protein 17B (Fragment),Mus musculus,10090,Akap17b,1,1,NaN,MFDTTKHFSEGAIQRRNQERLKLQELEEERKKEKKREEEVAERKRKDEVRKAQEKRKKARDRRRALKERDRHRQRKEKVKTKPEARQELDSSGEWE...
94784,Mau2,tr,Z4YNA3,Z4YNA3_MOUSE,MAU2 chromatid cohesion factor homolog,Mus musculus,10090,Mau2,1,1,NaN,MAAQAAAAAQAAAAAQAAAAQAAQAEAAESWYLALLGFAEHFRTSSPPKIRLCVHCLQAVFPFKPPQRIEARTHLQLGSVLYHHTKNSEQARSHLE...


In [ ]:
df = pd.read_pickle(PROTEIN_SEQ_PATH)

print(df.shape)

ds = df['seq'].drop_duplicates()

print(ds.shape)

corpus = ds.tolist()

ngram_count = {}
step = 1
n = 1
while True:

    vectorizer = CountVectorizer(analyzer='char', ngram_range=(n,n))

    X = vectorizer.fit_transform(corpus)

    count_vector = X.sum(axis=0)

    is_ones = (count_vector == 1)

    unique_count = is_ones.sum()

    print(n, X.shape[1], unique_count, abs(X.shape[1] - unique_count))

    ngram_count[n] = {'features': X.shape[1], 'unique_features': unique_count}

    with open(f'ngram_protein_{step}.pkl', 'wb') as f:
        pickle.dump(ngram_count, f, protocol=pickle.HIGHEST_PROTOCOL)

    if is_ones.all():
        break

    n = n + step